In [26]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, parent_dir)

import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

from CurveDataFetcher import CurveDataFetcher
from CurveBuilder import get_spot_rates_fitter, reprice_bonds

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
curve_data_fetcher = CurveDataFetcher(use_ust_issue_date=True, debug_verbose=True)

2024-09-05 22:01:06,855 - DataFetcher.base - DEBUG - UST Auctions - Number of Links to Fetch: 2
2024-09-05 22:01:06,856 - DataFetcher.base - DEBUG - UST Auctions - Links: ['https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?page[number]=1&page[size]=10000', 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?page[number]=2&page[size]=10000']


In [28]:
as_of_date = datetime(2024, 8, 29)
quote_type = "bid"

curve_set_df = curve_data_fetcher.build_curve_set(
    as_of_date=as_of_date,
    sorted=True, 
    use_github=True, 
    # use_public_dotcom=True,
    include_off_the_run_number=True,
    market_cols_to_return=[f"{quote_type}_price", f"{quote_type}_yield"],
    calc_free_float=True,
)

# remove OTRs and first off the runs due to liquidity premium
filtered_curve_set_df = curve_set_df[(curve_set_df["rank"] != 0) & (curve_set_df["rank"] != 1)]

# remove TBills
filtered_curve_set_df = filtered_curve_set_df[
    filtered_curve_set_df["security_type"] != "Bill"
]

# remove low free float bonds (< $8bn)
filtered_curve_set_df = filtered_curve_set_df[filtered_curve_set_df["free_float"] > 8000] 

# filter bonds very close to maturity
filtered_curve_set_df = filtered_curve_set_df[filtered_curve_set_df["time_to_maturity"] > 0.25] 

# remove CTDs (this is a TODO)
# filtered_curve_set_df = filtered_curve_set_df[filtered_curve_set_df["is_ctd"] == False] 

# remove cusips trading special in repo (this is a TODO)
# filtered_curve_set_df = filtered_curve_set_df[filtered_curve_set_df["is_trading_special"] == False] 

filtered_curve_set_df

2024-09-05 22:01:11,594 - DataFetcher.base - DEBUG - SOMA Holdings - Valid SOMA Holding Dates: {datetime.datetime(2024, 8, 29, 0, 0): datetime.datetime(2024, 8, 28, 0, 0)}
2024-09-05 22:01:11,597 - DataFetcher.base - DEBUG - STRIPping - BDays: [datetime.datetime(2024, 7, 31, 0, 0), datetime.datetime(2024, 6, 30, 0, 0)]
2024-09-05 22:01:11,598 - DataFetcher.base - DEBUG - STRIPping - 2024-08-29 00:00:00 url: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/debt/mspd/mspd_table_5?filter=record_date:in:(2024-07-31,2024-06-30)&page[number]=1&page[size]=10000


Using SOMA Holdings Data As of 2024-08-28
Using STRIPping Data As of 2024-07-31 00:00:00


,cusip,security_type,auction_date,issue_date,maturity_date,time_to_maturity,int_rate,high_investment_rate,is_on_the_run,label,...,parValue,percentOutstanding,est_outstanding_amt,corpus_cusip,outstanding_amt,portion_unstripped_amt,portion_stripped_amt,reconstituted_amt,free_float,rank
53,91282CFX4,Note,2022-11-21,2022-11-30,2024-11-30,0.254795,4.500,NaN,False,"Nov 24s, 2-Year",...,4.566032e+09,0.098063,4.656210e+10,912821KD6,4.656210e+10,46551301.7,1.080000e+07,0.0,41985.2693,20.0
54,91282CGD7,Note,2022-12-27,2023-01-03,2024-12-31,0.339726,4.250,NaN,False,"Dec 24s, 2-Year",...,0.000000e+00,NaN,NaN,912821KH7,4.198899e+10,41988991.2,0.000000e+00,0.0,41988.9912,19.0
55,91282CGG0,Note,2023-01-24,2023-01-31,2025-01-31,0.424658,4.125,NaN,False,"Jan 25s, 2-Year",...,0.000000e+00,NaN,NaN,912821KN4,4.199199e+10,41991993.5,0.000000e+00,0.0,41991.9935,18.0
56,91282CGN5,Note,2023-02-21,2023-02-28,2025-02-28,0.501370,4.625,NaN,False,"Feb 25s, 2-Year",...,9.542573e+09,0.185187,5.152952e+10,912821KT1,5.152952e+10,51529515.1,0.000000e+00,0.0,41986.9417,17.0
57,91282CGU9,Note,2023-03-27,2023-03-31,2025-03-31,0.586301,3.875,NaN,False,"Mar 25s, 2-Year",...,0.000000e+00,NaN,NaN,912821KX2,4.199558e+10,41993978.2,1.600000e+06,0.0,41993.9782,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,912810TN8,Bond,2023-04-13,2023-04-17,2053-02-15,28.484932,3.625,NaN,False,"Feb 53s, 30-Year",...,9.367420e+09,0.141965,6.598403e+10,912803GP2,6.598403e+10,50170937.7,1.581309e+10,1534040.0,40803.5178,6.0
387,912810TR9,Bond,2023-07-13,2023-07-17,2053-05-15,28.728767,3.625,NaN,False,"May 53s, 30-Year",...,5.723292e+09,0.091437,6.259307e+10,912803GS6,6.259307e+10,39104686.8,2.348839e+10,2265540.0,33381.3949,5.0
388,912810TT5,Bond,2023-10-12,2023-10-16,2053-08-15,28.980822,4.125,NaN,False,"Aug 53s, 30-Year",...,8.606596e+09,0.120247,7.157430e+10,912803GU1,7.157430e+10,60802584.0,1.077172e+10,208800.0,52195.9878,4.0
389,912810TV0,Bond,2024-01-11,2024-01-16,2053-11-15,29.232877,4.750,NaN,False,"Nov 53s, 30-Year",...,4.567153e+08,0.006874,6.644364e+10,912803GW7,6.644364e+10,57766113.9,8.677531e+09,1851960.0,57309.3986,3.0


In [29]:
t2 = np.linspace(1/365, 30, 1000)

# fitted_zero_curve_func_ns, status_ns = calibrate_ns_ols(
#     filtered_curve_set_df["time_to_maturity"].to_numpy(),
#     filtered_curve_set_df[f"{quote_type}_yield"].to_numpy(),
# )
# assert status_ns

# fitted_zero_curve_func_nss, status_nss, lstsq_res_nss = calibrate_nss_ols(
#     filtered_curve_set_df["time_to_maturity"].to_numpy(),
#     filtered_curve_set_df[f"{quote_type}_yield"].to_numpy(),
#     tau0=(1, 3.5)
# )
# assert status_nss

# print(status_nss)
# print(lstsq_res_nss)

# fitted_zero_curve_func_dl, status_dl = calibrate_diebold_li_ols(
#     filtered_curve_set_df["time_to_maturity"].to_numpy(),
#     filtered_curve_set_df[f"{quote_type}_yield"].to_numpy(),
# )

# fitted_zero_curve_func_bc, status_bc = calibrate_bc_ols(
#     np.concatenate((np.array([1/365]), filtered_curve_set_df["time_to_maturity"].to_numpy())),
#     np.concatenate((np.array([5.31]), filtered_curve_set_df[f"{quote_type}_yield"].to_numpy())),
# )

# fitted_zero_curve_func_aug_bc, status_aug_bc = calibrate_bc_augmented_ols(
#     np.concatenate((np.array([1/365]), filtered_curve_set_df["time_to_maturity"].to_numpy())),
#     np.concatenate((np.array([5.31]), filtered_curve_set_df[f"{quote_type}_yield"].to_numpy())),
# )

# filtered_curve_set_mles_func, status_mles = calibrate_mles_ols(
#     filtered_curve_set_df["time_to_maturity"].to_numpy(),
#     filtered_curve_set_df[f"{quote_type}_yield"].to_numpy(),
#     overnight_rate=5.31,
#     N=6
# )


fitted_zero_curve_dict = get_spot_rates_fitter(
    curve_set_df=filtered_curve_set_df,
    as_of_date=as_of_date,
    on_rate=5.35,
    # ql_fitting_methods=["ql_f_ns", "ql_f_nss", "ql_f_es", "ql_f_sp", "ql_f_cbs"],
    ql_fitting_methods=["ql_f_cbs"],
    ql_zero_curve_interp_method="ql_z_interp_monot_cubic",
    daily_interpolation=True,
    # simple_poly=3,
    knots=[-20, -10, -5, 0, 1, 2, 3, 5, 7, 10, 15, 20, 25, 30, 35, 40, 45, 50]
)

In [30]:
rv_df = reprice_bonds(
    as_of_date=as_of_date,
    ql_zero_curves={
        # "ns": fitted_zero_curve_dict["ql_f_ns"]["ql_zero_curve"],
        # "nss": fitted_zero_curve_dict["ql_f_nss"]["ql_zero_curve"],
        # "es": fitted_zero_curve_dict["ql_f_es"]["ql_zero_curve"],
        "cbs": fitted_zero_curve_dict["ql_f_cbs"]["ql_zero_curve"],
    },
    curve_set_df=curve_set_df,
)
rv_df

91282CFG1 degenerate single date (August 30th, 2024) schedule
 seed date: August 31st, 2024
 exit date: August 30th, 2024
 effective date: August 30th, 2024
 first date: null date
 next to last date: null date
 termination date: August 31st, 2024
 generation rule: Backward
 end of month: 0
912828YE4 degenerate single date (August 30th, 2024) schedule
 seed date: August 31st, 2024
 exit date: August 30th, 2024
 effective date: August 30th, 2024
 first date: null date
 next to last date: null date
 termination date: August 31st, 2024
 generation rule: Backward
 end of month: 0
9128282U3 degenerate single date (August 30th, 2024) schedule
 seed date: August 31st, 2024
 exit date: August 30th, 2024
 effective date: August 30th, 2024
 first date: null date
 next to last date: null date
 termination date: August 31st, 2024
 generation rule: Backward
 end of month: 0


,cusip,label,issue_date,maturity_date,time_to_maturity,high_investment_rate,int_rate,rank,outstanding,soma_holdings,stripping_amount,free_float,bid_yield,bid_price,accured,cbs_repriced_npv,cbs_repriced_ytm,cbs_price_spread,cbs_ytm_spread
0,912797LA3,"Sep 24s, 17-Week",2024-08-06,2024-09-03,0.013699,5.381000,NaN,16.0,0.000000e+00,6.784389e+08,0.000000e+00,-678.4389,6.798202,99.927361,0.000000,99.941608,5.416506,-0.014247,138.169620
1,912797LG0,"Sep 24s, 17-Week",2024-08-13,2024-09-10,0.032877,5.381000,NaN,15.0,0.000000e+00,6.873750e+08,0.000000e+00,-687.3750,5.934867,99.825333,0.000000,99.840236,5.391423,-0.014903,54.344353
2,912797LH8,"Sep 24s, 17-Week",2024-08-20,2024-09-17,0.052055,5.355000,NaN,14.0,0.000000e+00,7.210193e+08,0.000000e+00,-721.0193,5.753885,99.722917,0.000000,99.739890,5.366540,-0.016973,38.734468
3,912797LJ4,"Sep 24s, 17-Week",2024-08-27,2024-09-24,0.071233,5.335000,NaN,13.0,0.000000e+00,7.426366e+08,0.000000e+00,-742.6366,5.626962,99.623722,0.000000,99.640554,5.341855,-0.016832,28.510667
4,912797LK1,"Oct 24s, 17-Week",2024-08-06,2024-10-01,0.090411,5.346000,NaN,12.0,0.000000e+00,5.076268e+08,0.000000e+00,-507.6268,5.496958,99.527000,0.000000,99.542217,5.317372,-0.015217,17.958529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,912810TT5,"Aug 53s, 30-Year",2023-10-16,2053-08-15,28.980822,NaN,4.125,4.0,7.157430e+10,8.606596e+09,1.077172e+10,52195.9878,4.190369,98.906250,0.168139,98.911071,4.190360,0.163318,0.000898
386,912810TV0,"Nov 53s, 30-Year",2024-01-16,2053-11-15,29.232877,NaN,4.750,3.0,6.644364e+10,4.567153e+08,8.677531e+09,57309.3986,4.170675,109.718750,1.381114,109.579293,4.179494,1.520571,-0.881963
387,912810TX6,"Feb 54s, 30-Year",2024-04-15,2054-02-15,29.484932,NaN,4.250,2.0,7.119879e+10,2.211754e+09,1.127392e+10,57713.1192,4.183023,101.125000,0.173234,101.381859,4.168205,-0.083625,1.481788
388,912810UA4,"May 54s, 30-Year",2024-07-15,2054-05-15,29.728767,NaN,4.625,1.0,7.642080e+10,7.428279e+09,5.191628e+09,63800.8965,4.172348,107.656250,0.578125,107.935197,4.157686,0.299178,1.466111


In [31]:
fig = px.scatter(
    curve_set_df,
    x="time_to_maturity",
    y=f"{quote_type}_yield",
    color="original_security_term",
    hover_data=["label"],
    labels={"time_to_maturity": "Time to Maturity (Years)", "yield": "Yield (%)"},
    title="Filtered USTs Curve Set"
)

# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_ns(t2), mode="lines", name="Nelson-Siegel"
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_nss(t2), mode="lines", name="Svensson"
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_dl(t2), mode="lines", name="Diebold-Li"
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_bc(t2), mode="lines", name="Björk-Christensen"
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_aug_bc(t2), mode="lines", name="Augmented Björk-Christensen"
#     )
# )

# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_dict["ql_f_nss"]["zero_interp_func"](t2), mode="lines", name="Fitted NSS - Zero Curve"
#     )
# )
fig.add_trace(
    go.Scatter(
        x=t2, y=fitted_zero_curve_dict["ql_f_cbs"]["zero_interp_func"](t2), mode="lines", name="Fitted Zero Curve"
    )
)

fig.update_layout(
    legend_title_text='Label',
    xaxis_title="Time to Maturity (Years)",
    yaxis_title=f"YTMs: {quote_type}_yield",
    width=1600,
    height=800,
    template="plotly_dark",
)

fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikethickness=1)
fig.show()